In [14]:
# Other requirements
import glob
import itertools
import json
import os
import re
import sys
import time
from collections import OrderedDict
from datetime import datetime, timedelta
from pathlib import Path
from bs4 import BeautifulSoup


import astral
import numpy as np
import pandas as pd
import plotly as py
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import toml
import xarray as xr
from IPython.display import Image
from plotly.subplots import make_subplots
from tqdm.notebook import tqdm

xr.set_options(keep_attrs=True)

In [15]:
def rebase_path(path_base="urbisphere-dm", path_root=None):
    """return abs path of a higher level directory"""
    from pathlib import Path

    path_root = Path("__file__").parent.resolve() if not path_root else path_root
    path_parts = lambda p: p[0 : (p.index(path_base) + 1 if path_base in p else len(p))]
    return str(Path(*[n for n in path_parts(Path(path_root).parts)]))

In [ ]:
def urbisphere_colorbar(
    varname="wind speed",
    colorramp_file=dict(
        base_path=os.path.join(
            rebase_path(), "common/colormap/data/"
        ),  # "../tools/colormap/data/",
        file_pattern="ColorRampsWetterstation.nc",
    ),
):
    cm = xr.open_dataset(os.path.join(*colorramp_file.values()))
    # print(cm)
    cm_bar = []
    cm_lim = []
    cm_lab = set()
    for ind1, grp1 in cm.sel(name=varname).groupby("index"):
        cm_lim = grp1["colorscale_limits"].values.tolist()
        for ind2, grp2 in grp1.groupby("bounds"):
            cm_bar.append(
                (
                    grp2["colorscale_bounds"].values.tolist(),
                    grp2["RGB_label"].values.tolist(),
                )
            )
            cv_lim = grp2["variable_bounds"].values.tolist()
            if (cv_lim >= cm_lim[0]) and (cv_lim <= cm_lim[1]):
                cm_lab.add((grp2["variable_bounds"].values.tolist()))
        # print(grp1)

    return {
        "color_continuous_scale": cm_bar,
        "range_color": cm_lim,
        "tick_values": cm_lab,
    }

In [ ]:
def colorbar_beta(
    name="DWL",
    a=100,
    b=0,
    range_color=None,
    color_continuous_scale=None,
    color_discrete=False,
):
    # zat = unique( c(seq(5,50,by=15),seq(50,100,by=50),seq(100,500,by=100),seq(500,3000,by=500),seq(5000,30000,by=5000)))
    # zat = unique( c(seq(0,50,by=5),seq(50,100,by=50),seq(100,500,by=100),seq(500,3000,by=500),seq(5000,10000,by=5000)))

    zat = np.concatenate(
        [
            np.array([5]),
            np.array([100, 300, 500]),
            np.array([1000, 3000, 5000]),
            np.array([10000, 30000, 50000]),
            np.array([100000, 300000, 500000]),
        ]
    )

    def exp_text(n, a=1):
        if a == 1:
            a = ""
        else:
            a = "{:.0f} ".format(a)
        return "{a}10<sup>{n}</sub>".format(a=a, n=n)

    if name in ["DWL"]:
        zat = np.log10(1 / (zat[::-1] * a))
        zat_text = {n: exp_text(str(n)) for n in [-3, -4, -5, -6, -7, -8]}
    elif name in ["ALC"]:
        zat = np.log10(zat * a)
        zat_text = {n: exp_text(str(n)) for n in [1, 2, 3, 4, 5, 6, 7, 8, 9]}

    zat = zat.tolist()
    zat_vals = zat
    zat_text = [zat_text[n] if n in zat_text else "" for n in zat_vals]

    # range
    if not range_color:
        range_color = [np.min(zat), np.max(zat)]
    else:
        if range_color[0] != np.min(zat):
            zat = [range_color[0]] + zat
        if range_color[1] != np.max(zat):
            zat = zat + [range_color[1]]

        # - [ ] remove out of range values

    # colors
    if not color_continuous_scale:
        ccs = px.colors.colorbrewer.Spectral
        ccs = ccs
        ccs = ccs + [
            px.colors.colorbrewer.Greys_r[2 + n] for n in range(len(zat) - len(ccs))
        ]
        ccs = ccs[::-1]
        scs = (np.sort(np.array(zat)) - range_color[0]) / (
            range_color[1] - range_color[0]
        )
        color_continuous_scale = px.colors.make_colorscale(ccs, scs)

    # discretise
    if color_discrete:
        k = None
        iccs = []
        for i in range(0, len(color_continuous_scale) - 1):
            k, l = color_continuous_scale[i]
            m, n = color_continuous_scale[i + 1]
            iccs.append([k, n])
            iccs.append([m, n])

        color_continuous_scale = iccs

    return {
        "tickvals": zat_vals,
        "ticktext": zat_text,
        "range_color": range_color,
        "color_continuous_scale": color_continuous_scale,
    }


def colorbar_beta_reset(
    csb,
    bins=[2.0, 5.0, 6.0, 7.0, 8.0],
    colors=[
        "rgb(250,250,250)",
        "rgb(250,250,250)",
        "rgb(225,225,225)",
        "rgb(127,127,127)",
        "rgb(127,127,127)",
        "rgb(127,127,127)",
    ],
    reverse=False,
):
    ccc = [n[0] for n in csb["color_continuous_scale"]]
    cct = [csb["range_color"][0]] + csb["tickvals"] + [csb["range_color"][1]]
    ccb = np.digitize(cct[::-1], bins, right=False)
    if reverse:
        colors = colors[::-1]
    ccv = [colors[n] for n in ccb]
    return {
        **csb,
        **{"color_continuous_scale": px.colors.make_colorscale(ccv[::-1], ccc)},
    }


def colorbar_linear_depol_ratio(
    range_color=None,
    color_continuous_scale=None,
    color_discrete=False,
):
    from decimal import Decimal

    # water particles
    tick_config = {
        "Water": [
            dict(label="Fog", range=[0.00, 0.01]),
            dict(label="Rain", range=[0.01, 0.20]),
            dict(label="Snow", range=[0.20, 0.40]),
            dict(label="Ice", range=[0.40, 0.60]),
        ],
        "Other": [
            dict(label="Smoke", range=[0.01, 0.10]),
            dict(label="Dust", range=[0.10, 0.20]),
            dict(label="Desert", range=[0.2, 0.35]),
            dict(label="Volcanic", range=[0.30, 0.5]),
        ],
    }

    def tick_text(n, a=0.1):

        if Decimal(str(n)) % Decimal(str(a)) != 0.0:
            n = ""
        else:
            n = "{:.2f} ".format(n)
        return "{n}".format(n=n)

    zat = [
        -0.05,
        -0.00001,
        0.0,
        0.005,
        0.02,
        0.05,
        0.10,
        0.15,
        0.20,
        0.25,
        0.30,
        0.40,
        0.65,
        0.70,
        0.700001,
        0.75,
    ]
    zat_vals = [n for n in zat if Decimal(str(n)) % Decimal(str(0.1)) == 0.0]
    zat_text = {n: tick_text(n) for n in zat_vals}

    # range
    if not range_color:
        range_color = [np.min(zat), np.max(zat)]
    else:
        if range_color[0] != np.min(zat):
            zat = [range_color[0]] + zat
        if range_color[1] != np.max(zat):
            zat = zat + [range_color[1]]

    # colors
    if not color_continuous_scale:
        ccs = (
            ["rgb(250,250,250)"] * 2
            + px.colors.colorbrewer.Spectral
            + ["rgb(250,250,250)"] * 3
        )
        if len(ccs) > len(zat):
            ccs = ccs[-len(zat) :]
        elif len(ccs) < len(zat):
            ccs = ccs + [
                px.colors.colorbrewer.Greys_r[3 + n] for n in range(len(zat) - len(ccs))
            ]
        ccs = ccs[::-1]
        scs = (np.sort(np.array(zat)) - range_color[0]) / (
            range_color[1] - range_color[0]
        )
        color_continuous_scale = px.colors.make_colorscale(ccs, scs)

    # discretise
    if color_discrete:
        k = None
        iccs = []
        for i in range(0, len(color_continuous_scale) - 1):
            k, l = color_continuous_scale[i]
            m, n = color_continuous_scale[i + 1]
            iccs.append([k, n])
            iccs.append([m, n])

        color_continuous_scale = iccs

    return {
        "tickvals": list(zat_text.keys()),
        "ticktext": list(zat_text.values()),
        "range_color": range_color,
        "color_continuous_scale": color_continuous_scale,
    }

In [ ]:
def plot_time(time_window="24H", time_start=None, time_stop=None):
    """Determine plotting bounds for dimension time"""
    if not time_start and time_stop:
        latest = pd.to_datetime(time_stop)
        oldest = latest - pd.Timedelta(time_window)
    elif time_start and not time_stop:
        oldest = pd.to_datetime(time_start)
        latest = oldest + pd.Timedelta(time_window)
    else:
        oldest = None
        latest = None

    return slice(oldest, latest)

In [ ]:
def plot_montage(
    fg,
    figure_layout={},
    image_layout={"scale": 1},
    montage_layout={"padding_width": 5},
):
    """Concatenate multiple plotly figures as single image montage."""
    import io

    import imageio as iio
    import skimage

    # tile images
    img_list = [
        skimage.io.imread(
            io.BytesIO(x.update_layout(**figure_layout).to_image(**image_layout))
        )
        for x in fg
    ]

    # adjust color
    img_list = [skimage.color.rgba2rgb(x) for x in img_list]

    # adjust size (padding)
    psize_list = [list(x.shape) for x in img_list]
    psize_max = np.max(psize_list, axis=0)
    vtrim = [0, 0]
    cval = np.mean(img_list[-1][0, 0, :])
    pimg_list = []
    for n, img in enumerate(img_list):
        height, width, channels = (
            np.array(psize_max) - np.array(psize_list[n])
        ).tolist()

        if n == 0:
            pimg = np.pad(img, ((height, 0), (0, width), (0, 0)), constant_values=cval)
            vtrim[0] = height
        elif n == (len(img_list) - 1):
            pimg = np.pad(img, ((0, height), (0, width), (0, 0)), constant_values=cval)
            vtrim[1] = -height
        else:
            pimg = skimage.transform.resize(
                img, psize_max, mode="constant", clip=True, cval=cval
            )
        pimg_list.append(pimg)

    # concatenate (montage)
    abc = np.array(pimg_list)
    if not "fill" in montage_layout:
        montage_layout["fill"] = (cval, cval, cval)
    img = skimage.util.montage(
        abc, channel_axis=3, grid_shape=(len(img_list), 1), **montage_layout
    )

    # trim padded header and footer
    img = img[
        slice(
            np.max([0, vtrim[0] - montage_layout["padding_width"]]),
            np.min([0, vtrim[1] + montage_layout["padding_width"]]),
        ),
        :,
        :,
    ]

    # save as png (in-memory)
    with io.BytesIO() as file_obj:
        img = (img * 255).astype(np.uint8)
        iio.imsave(file_obj, img, format="png")  # img is an X,Y,4 array RGBA 0-255.
        file_obj.seek(0)
        png = file_obj.read()

    return png


def get_gattrs_roles(contributor_list):
    """Create a dict lookup table for roles and names in DataSet global attributes 'author' and 'contributor'"""
    from collections import defaultdict

    s3 = defaultdict(set)
    for s in contributor_list:
        for m in re.finditer(r"(?P<fullname>.*?)\s\[(?P<role>.*?)\]", s):
            k = m.group("role")
            v = m.group("fullname")
            s3[k].update([v])
    return s3


def get_table_colors(template):
    """Return a color settings dict that should overwride the plotly template."""
    if template.endswith("dark"):
        colors = {
            "header": dict(line_color="black", fill_color="black", font_color="white"),
            "cell": dict(
                line_color="black", fill_color=["black", "#333"], font_color="white"
            ),
        }
    else:
        colors = {
            "header": dict(line_color="white", fill_color="white", font_color="black"),
            "cell": dict(
                line_color="white", fill_color=["white", "#eee"], font_color="black"
            ),
        }
    return colors


def plot_attrs_table(
    attrs_dict,
    keys=["title", "author / contributor / <b>contact</b>"],
    template="plotly",
):
    """Prepare tables for gattrs or new key-value attributes"""
    colors = get_table_colors(template)
    values = []
    for k in keys:
        if k in ['title']:
            val = str(attrs_dict[k])
            # trim
            if len(val)>130:
                val = val[0:130] + "..."
        elif k in ["creator or contributor"]: # 'author / <b>PI</b> / <i>contact</i>'
            s1 = re.sub(r"\s\[.*?\]", "", attrs_dict["author"]).split("; ")
            s1 = list(dict.fromkeys(s1))
            s2 = re.sub(r"\s\[.*?\]", "", attrs_dict["contributor"]).split("; ")
            s2 = list(dict.fromkeys([s1[0]] + s2))
            s2 = [BeautifulSoup(n, "lxml").text for n in s2] # remove any html tags
            d2 = get_gattrs_roles(attrs_dict["contributor"].split("; "))
            
            val = []
            for n in s2:
                if n in s1:
                    n = "<i>{}</i>".format(n)
                if n in d2["Principal Investigator"]:
                    n = "<b>{}</b>".format(n)
                val.append(n)
            val = "; ".join(val)
        elif k in ["production level / info"]:
            s1 = "".join(
                [attrs_dict[k] for k in ["production_level"] if k in attrs_dict]
            )
            s2 = "".join(
                [
                    attrs_dict[k]
                    for k in ["production_info", "production_profile"]
                    if k in attrs_dict
                ]
            )
            val = "<b>Level {}</b> / {}".format(s1, s2)
        elif k in ["version / creation time"]:
            s1 = "".join(
                [
                    attrs_dict[k]
                    for k in ["version", "production_version"]
                    if k in attrs_dict
                ]
            )
            s2 = "".join(
                [
                    attrs_dict[k]
                    for k in ["creation_time", "production_time"]
                    if k in attrs_dict
                ]
            )
            val = "<b>{}</b> / {}".format(s1, s2)
        elif k in ["<i>urbisphere resource</i>"]:
            s1 = [
                "<i>urbisphere</i> data management policy (DMP)",
                ", ",
                "incidental findings policy (IFP)",
                " and " "publication code of conduct (PCC)",
                " apply.",
            ]
            val = "{}".format("".join(s1))
        elif k in attrs_dict:
            val = str(attrs_dict[k])        
        values.append(val)
    values = [keys, values]

    header = dict(
        values=[["<b></b>"], ["<b></b>"]],  # Data Attributes
        align=["right", "left"],
        font=dict(size=10),
        height=0,
        **colors["header"]
    )
    cells = dict(
        values=values,
        align=["right", "left"],
        font_size=10,
        height=24,
        **colors["cell"]
    )
    return (header, cells)


def plot_header(
    attrs_dict,
    title="archive » <i>urbisphere</i>",
    title_alt="",
    template="plotly",
    width=1870,
):

    # Constants
    # set or determine dimensions
    logo_width = 0
    plot_width = 750 + 650 + 180 + 170
    plus_width = width - plot_width - logo_width
    plot_height = 55
    scale_factor = 1

    scatter_trace1 = dict(
        type="scatter",
        x=[0, plot_width * scale_factor],
        y=[0, plot_width * scale_factor],
        mode="markers",
        marker_opacity=0,
        name="title_bg",
    )

    header1, cells1 = plot_attrs_table(
        attrs_dict, keys=["<i>urbisphere resource</i>"], template=template
    )
    table_trace2 = dict(
        type="table",
        domain=dict(x=[0.55, 1], y=[0, 1]),
        columnorder=[1, 2],
        columnwidth=[120, 550],
        header=header1,
        cells=cells1,
    )

    fig = go.Figure(data=[scatter_trace1, table_trace2])

    # Add invisible scatter trace.
    # This trace is added to help the autoresize logic work.

    # Configure axes
    fig.update_layout(
        xaxis=dict(
            visible=False, range=[0, plot_height * scale_factor], domain=[0, 0.55]
        ),
    )

    fig.update_yaxes(
        visible=False,
        range=[0, plot_height * scale_factor],
        # the scaleanchor attribute ensures that the aspect ratio stays constant
        scaleanchor="x",
    )

    # set figure layout
    fig.update_layout(
        title={"text": "", "font": {"size": 14}},
        margin={"l": logo_width, "r": plus_width, "b": 10, "t": 0},
        height=plot_height,
        width=width,
        template=template,
        plot_bgcolor="#333" if template.endswith("dark") else "#cee5fa",
    )

    ann_cfg = dict(
        showarrow=False,
        xref="paper",
        yref="paper",
        yanchor="top",
        xanchor="left",
        font_size=20,
    )

    fig.add_annotation(
        text=title,
        **{
            **ann_cfg,
            **dict(
                x=0,
                y=1,
                xshift=10,
                yshift=-7,
            ),
        }
    )
    fig.add_annotation(
        text=title_alt,
        **{
            **ann_cfg,
            **dict(
                x=0.55,
                y=1,
                xanchor="right",
                xshift=-10,
                yshift=-7,
            ),
        }
    )
    return fig


def plot_footer(attrs_dict, template="plotly", width=1870):

    # prepare table data
    header1, cells1 = plot_attrs_table(
        attrs_dict,
        keys=["title", "creator or contributor"],  # 'author / <b>PI</b> / <i>contact</i>'
        template=template,
    )
    header2, cells2 = plot_attrs_table(
        attrs_dict,
        keys=["production level / info", "version / creation time"],
        template=template,
    )

    # set or determine dimensions
    logo_width = 180
    plot_width = 750 + 650 + 170
    plus_width = width - plot_width - logo_width
    plot_height = 70

    # evaluate multiline footer for authors:
    if any([len(n) > 160 for v in cells1["values"] for n in v]):
        header1["height"] = header1["height"] + 15
        # header2['height'] = header2['height'] + 15
        plot_height = plot_height + 15

    # generate traces
    table_trace1 = dict(
        type="table",
        domain=dict(x=[0.0, 0.55], y=[0, 1.0]),
        columnorder=[1, 2],
        columnwidth=[70, 500],
        header=header1,
        cells=cells1,
    )
    table_trace2 = dict(
        type="table",
        domain=dict(x=[0.55, 1.0], y=[0, 1.0]),
        columnorder=[1, 2],
        columnwidth=[170, 500],
        header=header2,
        cells=cells2,
    )

    # generate figure
    fig = go.Figure(data=[table_trace1, table_trace2])

    # set figure layout
    fig.update_layout(
        title={"text": "", "font": {"size": 14}},
        margin={"l": logo_width, "r": plus_width, "b": 0, "t": 0},
        height=plot_height,
        width=width,
        template=template,
    )

    # add images as layout overlays
    fig.add_layout_image(
        dict(
            source=os.path.join(
                rebase_path(), "common/plotly/src/assets/", "urbisphere-logo.png"
            ),
            xref="paper",
            yref="paper",
            x=0,
            y=0,
            sizex=logo_width / (plot_width),
            sizey=1,
            xanchor="right",
            yanchor="bottom",
        )
    )

    return fig

In [ ]:
def fig_scale(fig, width=1870):
    lay = fig.to_dict()["layout"]
    r = default_width / lay["width"]
    height = np.round(r * lay["height"])
    fig = fig.update_layout(height=height, width=width)
    return fig

In [ ]:
# convert back to pandas when more convenient...
def urbisphere_stats_plot_timeheight(
    dx_vad,
    system_id="204",
    time_window="24H",
    time_start=None,
):
    """Plot"""
    ann_time_format = "%Y-%m-%d %H:%M:%S"
    ann_config = dict(xref="paper", yref="paper", showarrow=False)

    scale_config = {
        "CHM 15k": dict(name="ALC", range_color=[2, 8], a=100),
        "CL31": dict(name="ALC", range_color=[-2, 6], a=1),
        "CL61": dict(name="DWL", range_color=[-8, -2], a=100),
        "StreamLine": dict(name="DWL", range_color=[-8, -2], a=100),
        "StreamLine XR": dict(name="DWL", range_color=[-8, -2], a=100),        
    }
    rescale_config = {
        "CHM 15k": dict(
            bins=[2.0, 5.0, 6.0, 7.0, 8.0],
        ),
        "CL31": dict(
            bins=[-2.0, 3.0, 4.0, 5.0, 6.0],
        ),
        "CL61": dict(
            bins=[-8, -5, -4, -3, -2],
            reverse=False,
        ),
        "StreamLine": dict(
            bins=[-8, -5, -4, -3, -2],
            reverse=False,
        ),
        "StreamLine XR": dict(
            bins=[-8, -5, -4, -3, -2],
            reverse=False,
        ),        
    }

    panel_configs = {
        "DWL": {
            "t": {
                "height": 90,
                "range": [2000, 4000],
            },
            "b": {
                "height": 150,
                "range": [0, 2000],
            },
        },
        "ALC": {
            "t": {
                "height": 90,
                "range": [4000, 15000],
            },
            "b": {
                "height": 150,
                "range": [0, 4000],
            },
        },
    }

    system_group = "".join(dx_vad.coords["system_group"].values.tolist())
    sensor_name = "".join(dx_vad.coords["sensor_name"].values.tolist())

    scale_dict = scale_config[sensor_name]
    rescale_dict = rescale_config[sensor_name]

    plot_config = {
        "rcs_0": dict(
            color_continuous_scale=colorbar_beta(**scale_dict)[
                "color_continuous_scale"
            ],
            labels={"color": "rcs_0<br>({units})", "y": "Altitude (m AGL)"},
            range_color=colorbar_beta(**scale_dict)["range_color"],
            log_color=np.log10,
        ),
        "cloud_base_height": dict(
            category_orders={
                "channel": [
                    tuple([i, j])
                    for i, j in list(itertools.product(["cloud"], [1, 2, 3, 4, 5]))
                    + list(itertools.product(["aerosol"], [1, 2, 3, 4, 5]))
                ]
            },
            labels={"channel": "layer"},
            color_discrete_map={
                ("cloud", 1): "#e96088",
                ("cloud", 2): "#f19eb1",
                ("cloud", 3): "#f9d2da",
                ("cloud", 4): "#f9d2da",
                ("cloud", 5): "#f9d2da",
                ("aerosol", 1): "#0ee1f1",
                ("aerosol", 2): "#00b8ff",
                ("aerosol", 3): "#0083ff",
                ("aerosol", 4): "#005cff",
                ("aerosol", 5): "#0019ff",
            },
            symbol_map={"cloud": "triangle-up-open", "aerosol": "cross-open"},
            size_max=10,
        ),
        "linear_depol_ratio": dict(
            color_continuous_scale=colorbar_linear_depol_ratio()[
                "color_continuous_scale"
            ],
            labels={"color": "Linear<br>Depol<br>Ratio (-)", "y": "Altitude (m AGL)"},
            range_color=colorbar_linear_depol_ratio()["range_color"],
        ),
        "wdir": dict(
            color_continuous_scale="mygbm_r",
            labels={"color": "wdir<br>", "y": "Altitude (m AGL)"},
            range_color=[0, 360],
        ),
        "ws": dict(
            color_continuous_scale=urbisphere_colorbar(varname="wind speed")[
                "color_continuous_scale"
            ],
            labels={"color": "ws<br>(m s-1)", "y": "Altitude (m AGL)"},
            range_color=urbisphere_colorbar(varname="wind speed")["range_color"],
        ),
        "w_mean": dict(
            color_continuous_scale="rdbu_r",
            labels={"color": "w<br>(m s-1)", "y": "Altitude (m AGL)"},
            range_color=[-2, 2],
        ),
        "Beta_median": dict(
            color_continuous_scale=colorbar_beta(**scale_dict)[
                "color_continuous_scale"
            ],
            labels={"color": "beta att<br>(-)", "y": "Altitude (m AGL)"},
            range_color=colorbar_beta(**scale_dict)["range_color"],
            log_color=np.log10,
        ),
    }

    plot_layout = {
        "rcs_0": dict(
            coloraxis_colorbar=dict(
                tickvals=colorbar_beta(**scale_dict)["tickvals"],
                ticktext=colorbar_beta(**scale_dict)["ticktext"],
                ticks="outside",
                thicknessmode="pixels",
                thickness=20,
            )
        ),
        "cloud_base_height": {
            "legend": dict(
                itemsizing="constant",
                title={"text": "Layer<br>    (type, index)"},
                tracegroupgap=0,
                x=1.005,
            ),
        },
        "linear_depol_ratio": dict(
            coloraxis_colorbar=dict(
                tickvals=colorbar_linear_depol_ratio()["tickvals"],
                ticktext=colorbar_linear_depol_ratio()["ticktext"],
                ticks="outside",
                thicknessmode="pixels",
                thickness=20,
            )
        ),
        "Beta_median": dict(
            coloraxis_colorbar=dict(
                tickvals=colorbar_beta(**scale_dict)["tickvals"],
                ticktext=colorbar_beta(**scale_dict)["ticktext"],
                ticks="outside",
                thicknessmode="pixels",
                thickness=20,
            )
        ),
        "default": dict(
            coloraxis_colorbar=dict(
                ticks="outside",
                thicknessmode="pixels",
                thickness=20,
                x=1.005,
            )
        ),
    }
    panel_config = panel_configs[system_group]

    figs = []

    if not time_start:
        time_slice = plot_time(
            time_stop=dx_vad.time.max().values, time_window=time_window
        )
        title_text = "Figure:\t {now}<br>   Data:\t {latest}"
    else:
        time_slice = plot_time(time_start=time_start, time_window=time_window)
        title_text = "{start}"

    meta_dict = (
        dx_vad.drop_dims(["cell", "time"])
        .coords.to_dataset()
        .to_dataframe()
        .iloc[0]
        .to_dict()
    )
    meta_text = "{station_name} ({station_id}, {system_group} {sensor_name} {sensor_id}) at {station_lat:.3f},{station_lon:.3f}"

    dx_vad_fig = (
        dx_vad.drop_duplicates("cell").sel(time=time_slice, system=system_id)
        # .isel(cell=slice(0, 199))
    )

    dx_fig = xr.Dataset(coords={"time": [time_slice.start, time_slice.stop]})
    dx_vad_fig = xr.merge([dx_vad_fig, dx_fig])

    ann_time = {
        "start": dx_fig.time.min().dt.strftime(ann_time_format).values,
        "now": datetime.now().strftime(ann_time_format),
        "latest": dx_vad.time.max().dt.strftime(ann_time_format).values,
    }

    panel_config = panel_configs[system_group]

    for k, kconfig in plot_config.items():
        if k in dx_vad_fig.keys():
            if k in ["cloud_base_height"]:

                def scatter_data(dxe, k="cloud_base_height"):
                    dk = (
                        dxe[k]
                        .set_index(channel=["channel_type", "channel_id"])
                        .to_pandas()
                    )
                    dk = pd.melt(
                        dk,
                        var_name=["channel_type", "channel_id"],
                        value_name=k,
                        ignore_index=False,
                    )
                    dk["channel"] = dk.apply(
                        lambda x: (x["channel_type"], x["channel_id"]), axis=1
                    )
                    dk = dk.reset_index()
                    return dk

                dk = scatter_data(dx_vad, k)
                tmp = px.scatter(
                    dk,
                    x="time",
                    y=k,
                    color=dk["channel"],
                    symbol=dk["channel_type"],
                    size=(5 - dk["channel_id"]) * 3,
                    **kconfig,
                )

                cax = colorbar_beta_reset(
                    colorbar_beta(**scale_dict),
                    **rescale_dict,
                )
                cax = colorbar_beta(
                    **scale_dict,
                    color_continuous_scale=cax["color_continuous_scale"],
                    color_discrete=True,
                )

                fig = go.Figure(fig_bg)
                fig.add_hline(
                    y=panel_config["b"]["range"][1],
                    line_width=1,
                    line_dash="dash",
                    line_color="black",
                    row=1,
                    col=1,
                )
                fig.add_hline(
                    y=panel_config["b"]["range"][1],
                    line_width=1,
                    line_dash="dash",
                    line_color="black",
                    row=2,
                    col=1,
                )
                for trace in tmp["data"]:
                    fig.add_trace(trace, row=1, col=1)
                    fig.add_trace(trace, row=2, col=1)
                    fig.update_traces(showlegend=False, row=2, col=1)

                fig.for_each_trace(
                    lambda t: (
                        t.update(
                            legendgroup=",".join(t["legendgroup"].split(",")[0:2]),
                            name=",".join(t["name"].split(",")[0:2]),
                        )
                        if t["type"] in ["scatter", "scattergl"]
                        else ()
                    )
                )

                fig.update_coloraxes(
                    colorscale=cax["color_continuous_scale"],
                    colorbar=dict(len=0.5, y=0, yanchor="bottom"),
                )
                if k in plot_layout.keys():
                    fig.update_layout(**plot_layout[k])

            else:
                df_vad_fig = dx_vad_fig[k].to_pandas().T.astype(float)
                config = kconfig.copy()
                config["labels"]["color"] = config["labels"]["color"].format(
                    **dx_vad_fig[k].attrs
                )
                config_height = np.sum([v["height"] for k, v in panel_config.items()])
                config_heights = [
                    panel_config["t"]["height"] / config_height,
                    panel_config["b"]["height"] / config_height,
                ]

                if "log_color" in config:
                    df_vad_fig = config["log_color"](df_vad_fig)
                    df_vad_fig = df_vad_fig.fillna(config["range_color"][0])
                    config.pop("log_color")

                if "range_color" in config:
                    df_vad_fig = df_vad_fig.clip(*config["range_color"])

                fig = make_subplots(
                    rows=2, cols=1, row_heights=config_heights, vertical_spacing=0
                )

                tmp = px.imshow(df_vad_fig, origin="lower", **config)

                fig.add_trace(tmp["data"][0], row=1, col=1)
                fig.add_trace(tmp["data"][0], row=2, col=1)

                fig.update_xaxes(
                    range=[time_slice.start, time_slice.stop],
                    title={"standoff": 0, "text": ""},
                )
                fig.update_layout(
                    title={"text": "", "font": {"size": 14}},
                    margin={"l": 10, "r": 120, "b": 0, "t": 15},
                    height=config_height,
                    width=800,
                )
                fig.update_xaxes(
                    ticks="outside",
                    tickfont=dict(size=11),
                    ticklabelmode="period",
                    # tickcolor="black",
                    ticklen=15,
                    dtick="D1",
                    tickformat="%a %e %b\n(doy %j)\n%Y",
                    minor=dict(
                        ticklen=6,
                        dtick=6 * 60 * 60 * 1000,
                        tick0="2016-07-03",
                        griddash="dot",
                        # gridcolor="white",
                    ),
                )
                fig.update_yaxes(
                    ticks="outside",
                    tickfont=dict(size=11),
                    ticklen=6,
                    title_standoff=25,
                    tick0=0,
                    dtick=1000,
                    tickformat="~s",
                    autorange=False,
                )

                fig.add_annotation(
                    x=0,
                    y=1,
                    align="right",
                    yanchor="bottom",
                    text=meta_text.format(**meta_dict),
                    **ann_config,
                )

                fig.update_yaxes(range=panel_config["b"]["range"], row=2, col=1)
                fig.update_yaxes(range=panel_config["t"]["range"], row=1, col=1)
                fig.add_hline(
                    y=panel_config["b"]["range"][1],
                    line_width=1,
                    line_dash="dash",
                    line_color="white",
                    row=1,
                    col=1,
                )
                fig.add_hline(
                    y=panel_config["b"]["range"][1],
                    line_width=1,
                    line_dash="dash",
                    line_color="white",
                    row=2,
                    col=1,
                )
                fig.update_xaxes(
                    side="top", showticklabels=False, visible=False, row=1, col=1
                )  #
                fig.update_layout(coloraxis=tmp["layout"]["coloraxis"])

                fig.update_layout(**plot_layout["default"])
                if k in plot_layout.keys():
                    fig.update_layout(**plot_layout[k])

                if k == "rcs_0":
                    fig_bg = go.Figure(fig)

            figs.append((k, fig))

    return tuple(figs)